# Kronecker-Factored Laplace Approximation

## Imports

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms

import numpy as np
import matplotlib.pyplot as plt

from typing import List, Dict, Tuple, Optional, Any
import time
import os
import copy
from tqdm import tqdm

from typing import Any, Dict, List, Optional, Tuple
import abc

In [2]:
# @title
class BaseBayesianEnsemble(abc.ABC):
    """Базовый класс для всех методов байесовского ансамблирования"""

    def __init__(self, model: nn.Module, **kwargs):
        self.model = model
        self.is_fitted = False

    @abc.abstractmethod
    def fit(self,
            train_loader: torch.utils.data.DataLoader,
            val_loader: Optional[torch.utils.data.DataLoader] = None,
            **kwargs) -> Dict[str, List[float]]:
        """Обучение ансамбля"""
        # Надо обсудить параметры и реализацию
        pass

    @abc.abstractmethod
    def predict(self,
                X: torch.Tensor,
                n_samples: int = 100) -> Tuple[torch.Tensor, torch.Tensor]:
        """Предсказание с оценкой неопределенности"""
        # Надо обсудить параметры и реализацию
        pass

    @abc.abstractmethod
    def sample_models(self, n_models: int = 10) -> List[nn.Module]:
        """Сэмплирование моделей из апостериорного распределения"""
        # Надо обсудить параметры
        pass

    def save(self, path: str):
        """Сохранение обученного ансамбля"""
        torch.save({
            'model_state_dict': self.model.state_dict(),
            'ensemble_state': self._get_ensemble_state(),
            'is_fitted': self.is_fitted
        }, path)

    def load(self, path: str):
        """Загрузка обученного ансамбля"""
        checkpoint = torch.load(path)
        self.model.load_state_dict(checkpoint['model_state_dict'])
        self._set_ensemble_state(checkpoint['ensemble_state'])
        self.is_fitted = checkpoint['is_fitted']

    @abc.abstractmethod
    def _get_ensemble_state(self) -> Dict[str, Any]:
        """Получение внутреннего состояния ансамбля"""
        # Надо обсудить параметры
        pass

    @abc.abstractmethod
    def _set_ensemble_state(self, state: Dict[str, Any]):
        """Установка внутреннего состояния ансамбля"""
        # Надо обсудить параметры
        pass

## Example model and training

In [3]:
# Neural Network Models
class SimpleNN(nn.Module):
    """Simple feedforward neural network for MNIST"""
    def __init__(self, input_dim=784, hidden_dims=[1200, 1200], output_dim=10, dropout_rate=0.0):
        super(SimpleNN, self).__init__()
        layers = []
        prev_dim = input_dim

        for i, hidden_dim in enumerate(hidden_dims):
            layers.append(nn.Linear(prev_dim, hidden_dim))
            layers.append(nn.ReLU())
            if dropout_rate > 0:
                layers.append(nn.Dropout(dropout_rate))
            prev_dim = hidden_dim

        layers.append(nn.Linear(prev_dim, output_dim))
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        return self.network(x)

In [4]:
# Training Function
def train_model(model, train_loader, val_loader, num_epochs=50, lr=1e-3, weight_decay=1e-4):
    """Train a neural network model"""
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    criterion = nn.CrossEntropyLoss()

    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []

    print("Starting training...")
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0

        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)

            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = output.max(1)
            train_total += target.size(0)
            train_correct += predicted.eq(target).sum().item()

            if batch_idx % 100 == 0:
                print(f'Epoch: {epoch}, Batch: {batch_idx}, Loss: {loss.item():.4f}')

        train_accuracy = 100. * train_correct / train_total
        train_losses.append(train_loss / len(train_loader))
        train_accuracies.append(train_accuracy)

        # Validation phase
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for data, target in val_loader:
                data, target = data.to(device), target.to(device)
                output = model(data)
                loss = criterion(output, target)

                val_loss += loss.item()
                _, predicted = output.max(1)
                val_total += target.size(0)
                val_correct += predicted.eq(target).sum().item()

        val_accuracy = 100. * val_correct / val_total
        val_losses.append(val_loss / len(val_loader))
        val_accuracies.append(val_accuracy)

        print(f'Epoch {epoch+1}/{num_epochs}:')
        print(f'  Train Loss: {train_losses[-1]:.4f}, Train Acc: {train_accuracy:.2f}%')
        print(f'  Val Loss: {val_losses[-1]:.4f}, Val Acc: {val_accuracy:.2f}%')
        print('-' * 50)

    return {
        'train_losses': train_losses,
        'val_losses': val_losses,
        'train_accuracies': train_accuracies,
        'val_accuracies': val_accuracies
    }

In [ ]:
"""Training pipeline"""
# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Data preparation
print("Loading and preparing data...")
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST('../data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('../data', train=False, transform=transform)

# Split training data for validation
train_size = int(0.9 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_subset, val_subset = torch.utils.data.random_split(train_dataset, [train_size, val_size])

train_loader = DataLoader(train_subset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=128, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

# Model training
print("\nTraining model...")
model = SimpleNN(input_dim=784, hidden_dims=[1200, 1200], output_dim=10)

training_history = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    num_epochs=2,
    lr=1e-3,
    weight_decay=1e-4
)

Loading and preparing data...


100%|██████████| 9.91M/9.91M [00:02<00:00, 4.68MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 114kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.17MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 10.2MB/s]



Training model...
Starting training...
Epoch: 0, Batch: 0, Loss: 2.3183
Epoch: 0, Batch: 100, Loss: 0.1952
Epoch: 0, Batch: 200, Loss: 0.1328
Epoch: 0, Batch: 300, Loss: 0.1700
Epoch: 0, Batch: 400, Loss: 0.0232
Epoch 1/2:
  Train Loss: 0.1978, Train Acc: 93.91%
  Val Loss: 0.1382, Val Acc: 96.08%
--------------------------------------------------
Epoch: 1, Batch: 0, Loss: 0.0723
Epoch: 1, Batch: 100, Loss: 0.0566
Epoch: 1, Batch: 200, Loss: 0.1414
Epoch: 1, Batch: 300, Loss: 0.1785
Epoch: 1, Batch: 400, Loss: 0.1211
Epoch 2/2:
  Train Loss: 0.0866, Train Acc: 97.32%
  Val Loss: 0.1327, Val Acc: 96.17%
--------------------------------------------------


## KFLA class

In [27]:
class LaplaceApproximation(BaseBayesianEnsemble):
    """
    A scalable Laplace approximation for neural networks based on Kronecker-factored curvature.
    """
    def __init__(
        self,
        model: nn.Module,
        pretrained: bool = True,
        likelihood: str = 'classification',
        verbose: bool = False,
    ):
        self.model = model
        self.is_fitted = False
        self.device = next(model.parameters()).device
        if likelihood in ['classification', 'regression']:
            self.likelihood = likelihood
        else:
            raise ValueError(f"Unsupported likelihood: {likelihood}")

        self.pretrained = pretrained

        # Storage for Kronecker factors
        self.kronecker_factors = {}
        self.sampling_factors = {}
        self.dataset_size = 1

        # Hook handles
        self.hook_handles = []

        self.verbose = verbose

    def toggle_verbose(self):
        """
        Turn verbose on or off.
        """
        self.verbose = not self.verbose
        print("Verbose:", "on" if self.verbose else "off")

    def fit(self,
            train_loader: torch.utils.data.DataLoader,
            val_loader: Optional[torch.utils.data.DataLoader] = None,
            num_epochs: int = 100,
            lr: float = 1e-3,
            prior_precision: float = 1.0,
            num_samples: int = 1000,
           ) -> Dict[str, List[float]]:

        history = {}

        if not self.pretrained:
            if self.verbose:
                print('Training model...')
            # Сначала обучаем модель MAP оценке
            optimizer = torch.optim.Adam(self.model.parameters(), lr=lr)
            history['train_loss'] = []

            for epoch in range(num_epochs):
                self.model.train()
                train_loss = 0.0
                for batch_X, batch_y in train_loader:
                    optimizer.zero_grad()
                    output = self.model(batch_X)
                    if self.likelihood == 'classification':
                        loss = F.cross_entropy(output, batch_y)
                    else:
                        loss = F.mse_loss(output, batch_y)
                    loss.backward()
                    optimizer.step()
                    train_loss += loss.item()

                total_loss = train_loss / len(train_loader)
                history['train_loss'].append(total_loss)

                if self.verbose:
                    print(f'Epoch {epoch}: training loss = {total_loss:.4f}')

            self.pretrained = True

        self.compute_posterior(train_loader, prior_precision, num_samples)

        return history

    def compute_posterior(self,
                         train_loader: DataLoader,
                         prior_precision: float = 0.0,
                         num_samples: int = 1000) -> None:
        """
        Compute the Kronecker-factored Laplace approximation.
        """
        self.prior_precision = prior_precision
        self.dataset_size = len(train_loader.dataset)

        if self.verbose:
            print("Registering hooks...")
        self._register_hooks()

        if self.verbose:
            print("Estimating Kronecker factors...")
        self._estimate_kronecker_factors(train_loader, num_samples)

        if self.verbose:
            print("Removing hooks...")
        self._remove_hooks()

        print("Posterior computation completed!")

    def _register_hooks(self) -> None:
        """Register forward hooks to capture activations and pre-activation Hessians."""
        self.activations = {}
        self.pre_activation_hessians = {}

        def make_forward_hook(layer_name):
            def forward_hook(module, input, output):
                if isinstance(input, tuple):
                    input = input[0]
                # Store input activations for Q factor
                self.activations[layer_name] = input.detach().clone()
            return forward_hook

        # Register hooks for linear layers
        for name, module in self.model.named_modules():
            if isinstance(module, nn.Linear):
                forward_handle = module.register_forward_hook(make_forward_hook(name))
                self.hook_handles.append(forward_handle)

    def _remove_hooks(self) -> None:
        """Remove all registered hooks."""
        for handle in self.hook_handles:
            handle.remove()
        self.hook_handles.clear()

    def _compute_pre_activation_hessian(self, output, target):
        """
        Compute the Hessian with respect to the final layer pre-activations.
        This depends on the likelihood function.
        """
        batch_size = output.shape[0]

        if self.likelihood == 'classification':
            # For cross-entropy loss with softmax, the Hessian is:
            # H = diag(p) - pp^T, where p is the softmax probabilities
            probs = F.softmax(output, dim=1)
            eye = torch.eye(probs.size(1), device=self.device).unsqueeze(0)  # (1, C, C)
            probs_outer = torch.einsum('bi,bj->bij', probs, probs)  # (B, C, C)
            hessian = probs_outer - eye  # This is actually -H, but we'll account for sign later
            return -hessian  # Return the negative Hessian of NLL

        else: # self.likelihood == 'regression':
            # For MSE loss, the Hessian is identity
            output_dim = output.shape[1]
            hessian = torch.eye(output_dim, device=self.device).unsqueeze(0).repeat(batch_size, 1, 1)
            return hessian


    def _backward_hessian(self, hessian_final):
        """
        Correct recursive backpropagation of pre-activation Hessian.
        Uses weights of the NEXT layer to compute Hessian for the CURRENT layer.
        """
        hessians = {}

        # Get all linear layers in forward order
        linear_layers = []
        for name, module in self.model.named_modules():
            if isinstance(module, nn.Linear):
                linear_layers.append((name, module))

        # Initialize with the final layer Hessian
        if hessian_final.dim() == 3:
            current_hessian = hessian_final.mean(0)  # Average over batch
        else:
            current_hessian = hessian_final

        final_layer_name, final_layer = linear_layers[-1]
        hessians[final_layer_name] = current_hessian

        # Backpropagate through layers in REVERSE order (from output to input)
        for i in range(len(linear_layers) - 2, -1, -1):  # Start from second-to-last layer
            current_layer_name, current_layer = linear_layers[i]
            next_layer_name, next_layer = linear_layers[i + 1]

            # The recursive formula: H_λ = W_{λ+1}^T @ H_{λ+1} @ W_{λ+1} + D_λ
            # For piecewise linear activations (ReLU), D_λ ≈ 0
            W_next = next_layer.weight  # Shape: (out_dim_{λ+1}, in_dim_{λ+1})

            # H_current = W_next^T @ H_next @ W_next
            H_current = W_next.T @ current_hessian @ W_next

            # Store the Hessian for this layer
            hessians[current_layer_name] = H_current

            # Update for next iteration
            current_hessian = H_current

        return hessians

    def _estimate_kronecker_factors(self, train_loader: DataLoader, num_samples: int) -> None:
        """
        Estimate Kronecker factors using proper Hessian computation.
        """
        self.model.eval()  # We want deterministic behavior for curvature estimation

        accumulators = {}
        sample_count = 0

        if self.verbose:
            print(f"Processing up to {num_samples} samples...")

        for batch_idx, (data, target) in enumerate(train_loader):
            if sample_count >= num_samples:
                break

            data, target = data.to(self.device), target.to(self.device)
            batch_size = data.shape[0]

            # Forward pass
            output = self.model(data)

            # Compute pre-activation Hessian for the final layer
            H_final = self._compute_pre_activation_hessian(output, target)

            # Backpropagate Hessian through all layers
            layer_hessians = self._backward_hessian(H_final)

            # Process each layer
            with torch.no_grad():
                for name, module in self.model.named_modules():
                    if isinstance(module, nn.Linear) and name in self.activations and name in layer_hessians:
                        a = self.activations[name]  # input activations (B, in_dim)
                        H = layer_hessians[name]    # pre-activation Hessian (out_dim, out_dim)

                        if name not in accumulators:
                            in_dim = a.shape[1]
                            out_dim = H.shape[0] if H.dim() == 2 else H.shape[1]

                            accumulators[name] = {
                                'Q_sum': torch.zeros(in_dim, in_dim, device=self.device),
                                'H_sum': torch.zeros(out_dim, out_dim, device=self.device),
                                'count': 0,
                                'in_dim': in_dim,
                                'out_dim': out_dim
                            }

                        # Compute Q factor: covariance of input activations
                        batch_Q = torch.einsum('bi,bj->ij', a, a) / batch_size

                        # H factor is already computed from backpropagation
                        # Average if we have multiple samples
                        if H.dim() == 3:  # batch of Hessians
                            batch_H = H.mean(0)
                        else:
                            batch_H = H

                        accumulators[name]['Q_sum'] += batch_Q * batch_size
                        accumulators[name]['H_sum'] += batch_H * batch_size
                        accumulators[name]['count'] += batch_size

                sample_count += batch_size
                if sample_count % 1000 == 0 and self.verbose:
                    print(f"Processed {sample_count} samples...")

        # Compute final factors with proper regularization
        if self.verbose:
            print("Computing final Kronecker factors...")
        with torch.no_grad():
            for name, acc in accumulators.items():
                if acc['count'] == 0:
                    continue

                # Expected Kronecker factors (Equation 7 in the paper)
                Q = acc['Q_sum'] / acc['count']  # E[Q_λ]
                H = acc['H_sum'] / acc['count']  # E[H_λ]

                # Add prior precision and scale by dataset size (Equation 9)
                N = self.dataset_size
                tau = self.prior_precision

                Q_reg = np.sqrt(N) * Q + np.sqrt(tau) * torch.eye(acc['in_dim'], device=self.device)
                H_reg = np.sqrt(N) * H + np.sqrt(tau) * torch.eye(acc['out_dim'], device=self.device)

                # Store the precision matrices for sampling
                self.kronecker_factors[name] = {
                    'Q': Q_reg,  # Precision for rows
                    'H': H_reg,  # Precision for columns
                }

                if self.verbose:
                    print(f"Layer {name}:\nQ shape {Q_reg.shape}, H shape {H_reg.shape}")
                    Q_norm = torch.norm(Q_reg).item()
                    H_norm = torch.norm(H_reg).item()
                    print(f"  Q norm: {Q_norm:.6f}, H norm: {H_norm:.6f}")
                    cond_Q = torch.linalg.cond(Q_reg)
                    cond_H = torch.linalg.cond(H_reg)
                    print(f"cond(Q)={cond_Q:.2e}, cond(H)={cond_H:.2e}")

                # Convert to covariance matrices for sampling
                U = torch.linalg.inv(Q_reg)  # Row covariance
                V = torch.linalg.inv(H_reg)  # Column covariance

                # Matrix sqrt for sampling
                L_U = self._matrix_sqrt(U)
                L_V = self._matrix_sqrt(V)

                self.sampling_factors[name] = {
                    'L_U': L_U,
                    'L_V': L_V,
                    'weight_shape': (acc['out_dim'], acc['in_dim'])
                }

    def _matrix_sqrt(self, A: torch.Tensor) -> torch.Tensor:
        """Compute matrix square root using eigen decomposition."""
        # Eigen decomposition for symmetric matrices
        L, V = torch.linalg.eigh(A)
        return V @ torch.diag(torch.sqrt(L)) @ V.T

    def sample_models(self, n_models: int = 10, temperature: float = 1.0) -> List[nn.Module]:
        """
        Sample weight matrices from the matrix normal posterior.
        Returns a list of dictionaries, where each dictionary contains
        the complete set of weights for one sampled model.

        Args:
            num_samples: Number of model samples to generate

        Returns:
            List of dictionaries, where each dict has layer names as keys
            and sampled weight tensors as values
        """
        samples = []

        for i in range(n_models):
            weight_sample = {}

            for name, factors in self.sampling_factors.items():
                module = dict(self.model.named_modules())[name]
                M = module.weight.data  # MAP estimate

                L_V = factors['L_V']  # Row precision (in_dim, in_dim)
                L_U = factors['L_U']  # Column precision (out_dim, out_dim)
                weight_shape = factors['weight_shape']

                # Generate single sample for this layer
                Z = torch.randn(weight_shape, device=self.device)
                W_sample = M + temperature * L_V @ Z @ L_U.T

                # Store in the model sample dictionary
                weight_sample[f"{name}.weight"] = W_sample.cpu()

            # Also include bias terms if they exist
            for name, module in self.model.named_modules():
                if isinstance(module, nn.Linear) and module.bias is not None:
                    # For bias, we can use a simple diagonal approximation
                    # or sample from the appropriate marginal distribution
                    if name in self.kronecker_factors:
                        # The bias is typically part of the same layer's distribution
                        # but for simplicity, we'll use the MAP estimate for bias
                        weight_sample[f"{name}.bias"] = module.bias.data.clone()

            model_sample = copy.deepcopy(self.model)
            model_sample.load_state_dict(weight_sample)

            samples.append(model_sample)

        return samples

    def predict(self,
                X: torch.Tensor,
                n_samples: int = 100,
                temperature: float = 1.0) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Compute predictive distribution using sampled weights.
        """
        predictions = []

        for _ in range(n_samples):
            sampled_model = self.sample_models(n_models=1, temperature=temperature)[0]
            sampled_model.to(self.device)

            with torch.no_grad():
                output = sampled_model(X)
                predictions.append(output)

            sampled_model.cpu()

        predictions = torch.stack(predictions)

        if self.likelihood == 'classification':
            probs = F.softmax(predictions, dim=-1)
            mean_probs = probs.mean(dim=0)
            uncertainty = -(mean_probs * torch.log(mean_probs + 1e-8)).sum(dim=-1)
            return mean_probs, uncertainty
        else:
            mean = predictions.mean(dim=0)
            variance = predictions.var(dim=0)
            return mean, variance

    def _get_ensemble_state(self) -> Dict[str, Any]:
        """Получение внутреннего состояния ансамбля"""
        return {
            'model': self.model,
            'is_fitted': self.is_fitted,
            'device': self.device,
            'likelihood': self.likelihood,
            'pretrained': self.pretrained,
            'kronecker_factors': self.kronecker_factors,
            'sampling_factors': self.sampling_factors,
            'dataset_size': self.dataset_size,
            'hook_handles': self.hook_handles,
            'verbose': self.verbose,
        }

    def _set_ensemble_state(self, state: Dict[str, Any]):
        """Установка внутреннего состояния ансамбля"""
        if state['likelihood'] in ['classification', 'regression']:
            self.likelihood = state['']
        else:
            raise ValueError(f"Unsupported likelihood: {state['']}")

        self.model = state['model']
        self.is_fitted = state['is_fitted']
        self.device = state['device']
        self.pretrained = state['pretrained']
        self.kronecker_factors = state['kronecker_factors']
        self.sampling_factors = state['sampling_factors']
        self.dataset_size = state['dataset_size']
        self.hook_handles = state['hook_handles']
        self.verbose = state['verbose']

In [28]:
# Let's test with extremely conservative settings
print("Creating KFLA...")

# Make sure the model is on the correct device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Create the Laplace approximation
laplace = LaplaceApproximation(model, likelihood='classification', verbose=True)

# Compute the posterior
print("Computing posterior...")
laplace.compute_posterior(
    train_loader=train_loader,
    prior_precision=1.0,
    num_samples=len(train_dataset)
)

Creating KFLA...
Computing posterior...
Registering hooks...
Estimating Kronecker factors...
Processing up to 60000 samples...
Processed 16000 samples...
Processed 32000 samples...
Processed 48000 samples...
Processed 54000 samples...
Computing final Kronecker factors...
Layer network.0:
Q shape torch.Size([784, 784]), H shape torch.Size([1200, 1200])
  Q norm: 62008.597656, H norm: 10311.433594
cond(Q)=5.81e+04, cond(H)=6.06e+03
Layer network.2:
Q shape torch.Size([1200, 1200]), H shape torch.Size([1200, 1200])
  Q norm: 29033.296875, H norm: 534.191406
cond(Q)=2.29e+04, cond(H)=2.50e+02
Layer network.4:
Q shape torch.Size([1200, 1200]), H shape torch.Size([10, 10])
  Q norm: 94298.226562, H norm: 667.978699
cond(Q)=7.99e+04, cond(H)=1.04e+00
Removing hooks...
Posterior computation completed!


## KFLA evaluation

In [29]:
# Evaluation functions
def evaluate_laplace_model(laplace, test_loader, num_samples=10, temperature=1.0):
    """Evaluate the model with Laplace approximation"""
    laplace.model.eval()
    correct = 0
    total = 0
    uncertainties = []

    with torch.no_grad():
        for data, target in tqdm(test_loader):
            data, target = data.to(device), target.to(device)

            mean_probs, uncertainty = laplace.predict(
                data, n_samples=num_samples, temperature=temperature
            )

            _, predicted = mean_probs.max(1)

            total += target.size(0)
            correct += predicted.eq(target).sum().item()
            uncertainties.extend(uncertainty.cpu().numpy())

    accuracy = 100. * correct / total
    avg_uncertainty = np.mean(uncertainties)

    return accuracy, avg_uncertainty, uncertainties

def evaluate_standard_model(model, test_loader):
    """Evaluate the standard model"""
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            _, predicted = output.max(1)
            total += target.size(0)
            correct += predicted.eq(target).sum().item()

    accuracy = 100. * correct / total
    return accuracy

In [13]:
# Test standard model first
print("\nEvaluating standard model...")
standard_accuracy = evaluate_standard_model(model, test_loader)
print(f"Standard Model Accuracy: {standard_accuracy:.2f}%")


Evaluating standard model...
Standard Model Accuracy: 95.91%


In [30]:
accuracy, uncertainty, _ = evaluate_laplace_model(
    laplace, test_loader, num_samples=10, temperature=1.0
)
print(f"10 samples, T = 1.0: Accuracy = {accuracy:.2f}%, Uncertainty = {uncertainty:.4f}")

100%|██████████| 79/79 [00:12<00:00,  6.35it/s]

10 samples, T = 1.0: Accuracy = 68.74%, Uncertainty = 1.2498


In [ ]:
for n_samples in [10, 20, 50, 100, 200]:
    accuracy, uncertainty, _ = evaluate_laplace_model(
        laplace, test_loader, num_samples=n_samples, temperature=1.0
    )
    print(f"{n_samples} samples, T = 1.0: Accuracy = {accuracy:.2f}%, Uncertainty = {uncertainty:.4f}")

10 samples, T = 1.0: Accuracy = 68.41%, Uncertainty = 1.2487
20 samples, T = 1.0: Accuracy = 73.72%, Uncertainty = 1.3669
50 samples, T = 1.0: Accuracy = 76.20%, Uncertainty = 1.4506
100 samples, T = 1.0: Accuracy = 77.43%, Uncertainty = 1.4796
200 samples, T = 1.0: Accuracy = 77.97%, Uncertainty = 1.4932


In [31]:
# Test Laplace with very conservative temperatures
print("\nTesting Laplace with different temperatures:")
temperatures = [0.001, 0.01, 0.1, 1, 10]

for temp in temperatures:
    accuracy, uncertainty, _ = evaluate_laplace_model(
        laplace, test_loader, num_samples=10, temperature=temp
    )
    print(f"Temperature {temp}: Accuracy = {accuracy:.2f}%, Uncertainty = {uncertainty:.4f}")


Testing Laplace with different temperatures:


100%|██████████| 79/79 [00:12<00:00,  6.46it/s]


Temperature 0.001: Accuracy = 95.91%, Uncertainty = 0.1048


100%|██████████| 79/79 [00:12<00:00,  6.54it/s]


Temperature 0.01: Accuracy = 95.92%, Uncertainty = 0.1048


...

Temperature 0.1: Accuracy = 95.90%, Uncertainty = 0.1059


100%|██████████| 79/79 [00:13<00:00,  5.87it/s]


Temperature 1: Accuracy = 69.61%, Uncertainty = 1.2507


100%|██████████| 79/79 [00:12<00:00,  6.48it/s]

Temperature 10: Accuracy = 10.36%, Uncertainty = 1.7648


In [38]:
# Let's also look at uncertainty for correct vs incorrect predictions
def analyze_uncertainty_by_accuracy(laplace, test_loader, num_samples=10):
    """Analyze uncertainty for correct vs incorrect predictions"""
    laplace.model.eval()
    correct_uncertainties = []
    incorrect_uncertainties = []

    with torch.no_grad():
        for data, target in tqdm(test_loader):
            data, target = data.to(device), target.to(device)

            # Get predictive distribution using Laplace approximation
            mean_probs, uncertainty = laplace.predict(data, n_samples=num_samples)

            # Get predictions
            _, predicted = mean_probs.max(1)

            # Separate uncertainties for correct and incorrect predictions
            correct_mask = predicted.eq(target)
            incorrect_mask = ~correct_mask

            if correct_mask.any():
                correct_uncertainties.extend(uncertainty[correct_mask].cpu().numpy())
            if incorrect_mask.any():
                incorrect_uncertainties.extend(uncertainty[incorrect_mask].cpu().numpy())

    return correct_uncertainties, incorrect_uncertainties

In [39]:
correct_unc, incorrect_unc = analyze_uncertainty_by_accuracy(laplace, test_loader)

print(f"\nUncertainty Analysis:")
print(f"Average uncertainty for correct predictions: {np.mean(correct_unc):.4f}")
print(f"Average uncertainty for incorrect predictions: {np.mean(incorrect_unc):.4f}")
print(f"Number of correct predictions: {len(correct_unc)}")
print(f"Number of incorrect predictions: {len(incorrect_unc)}")


Uncertainty Analysis:
Average uncertainty for correct predictions: 1.1289
Average uncertainty for incorrect predictions: 1.5216
Number of correct predictions: 6891
Number of incorrect predictions: 3109


In [41]:
# Example of getting predictions with uncertainty for a single batch
print("\nTesting on a single batch...")
data_iter = iter(test_loader)
test_data, test_target = next(data_iter)
test_data, test_target = test_data.to(device), test_target.to(device)

mean_probs, uncertainty = laplace.predict(test_data, n_samples=10)
_, predicted = mean_probs.max(1)

print(f"Predictions for first 10 test samples:")
for i in range(min(10, len(test_data))):
    print(f"  Sample {i+1}: True={test_target[i].item()}, Pred={predicted[i].item()}, "
          f"Uncertainty={uncertainty[i].item():.4f}, "
          f"Correct={predicted[i].eq(test_target[i]).item()}")


Testing on a single batch...
Predictions for first 10 test samples:
  Sample 1: True=7, Pred=7, Uncertainty=1.2757, Correct=True
  Sample 2: True=2, Pred=2, Uncertainty=1.3885, Correct=True
  Sample 3: True=1, Pred=1, Uncertainty=1.5347, Correct=True
  Sample 4: True=0, Pred=2, Uncertainty=1.5881, Correct=False
  Sample 5: True=4, Pred=9, Uncertainty=1.3691, Correct=False
  Sample 6: True=1, Pred=1, Uncertainty=0.6458, Correct=True
  Sample 7: True=4, Pred=5, Uncertainty=1.5984, Correct=False
  Sample 8: True=9, Pred=9, Uncertainty=1.6587, Correct=True
  Sample 9: True=5, Pred=5, Uncertainty=1.5540, Correct=True
  Sample 10: True=9, Pred=9, Uncertainty=1.1116, Correct=True


In [43]:
model_samples = laplace.sample_models(n_models=10)
model_samples[0]

SimpleNN(
  (network): Sequential(
    (0): Linear(in_features=784, out_features=1200, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1200, out_features=1200, bias=True)
    (3): ReLU()
    (4): Linear(in_features=1200, out_features=10, bias=True)
  )
)